# Adatok előkészítése - notebook

##### 1. Feladat:
Adatgyűjtés és előkészítés: Termékértékelési adatok letöltése egy nyílt adatforrásból (pl. Kaggle vagy Amazon API), és előfeldolgozás (szövegbeli zajok eltávolítása, tokenizálás stb.)

##### Adathalmaz elérési link: https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews?resource=download

## Szükséges könyvtárak importálása

In [ ]:
#!pip install --upgrade nltk

In [19]:
import os
import re

import numpy as np
import pandas as pd

import nltk

In [20]:
nltk.download('all')
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/vani/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /home/vani/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/vani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/vani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/vani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package ave

True

In [21]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer

In [22]:
!pip install nbformat
%run 'common.ipynb'

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/vani/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /home/vani/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/vani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/vani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/vani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package ave

## Adathalmazok betöltése, adatok megtekintése

C:\szoveg_web_banyaszat\adat

In [23]:
file = 'train.csv'
test_file = 'test.csv'

path = input('Path for the files: ')

file_path = (os.path.join(path, file)).replace('\\', '/')
test_file_path = (os.path.join(path, test_file)).replace('\\', '/')

In [24]:
data = pd.read_csv(file_path, names=['sentiment', 'title', 'text'])

In [25]:
data.head(10)

,sentiment,title,text
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
5,2,an absolute masterpiece,I am quite sure any of you actually taking the...
6,1,Buyer beware,"This is a self-published book, and if you want..."
7,2,Glorious story,I loved Whisper of the wicked saints. The stor...
8,2,A FIVE STAR BOOK,I just finished reading Whisper of the Wicked ...
9,2,Whispers of the Wicked Saints,This was a easy to read book that made me want...


In [26]:
data.shape

(3600000, 3)

In [27]:
data.sentiment.unique().tolist()

[2, 1]

In [28]:
data.sentiment.value_counts()

sentiment
2    1800000
1    1800000
Name: count, dtype: int64

In [29]:
data_test = pd.read_csv(test_file_path, names=['sentiment', 'title', 'text'])

In [30]:
data_test.head(10)

,sentiment,title,text
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
5,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...
6,1,Incorrect Disc,"I love the style of this, but after a couple y..."
7,1,DVD menu select problems,I cannot scroll through a DVD menu that is set...
8,2,Unique Weird Orientalia from the 1930's,"Exotic tales of the Orient from the 1930's. ""D..."
9,1,"Not an ""ultimate guide""","Firstly,I enjoyed the format and tone of the b..."


#### Nan értékek detektálása

In [31]:
data.isna().sum()

sentiment      0
title        207
text           0
dtype: int64

In [32]:
# Azon sorok kiíratása amik tartalamnak Nan értéket - mivel rengeteg adatunk van, ezt a 77 sort törölhetjük
data[data.isna().any(axis=1)]

,sentiment,title,text
13265,1,NaN,Couldn't get the device to work with my networ...
26554,1,NaN,What separates this band from Evanescence (bes...
26827,2,NaN,Falkenbach returns with more of the Viking/Fol...
36598,2,NaN,I returned this because I received the same on...
37347,2,NaN,This book is a great fantasy. I love this amaz...
...,...,...,...
3403351,1,NaN,It is not a game. It is only a memory cardIt w...
3455848,1,NaN,"The sleeve is not bad, but the vacuum is worth..."
3493132,2,NaN,Al Spath's diary is a must for all poker playe...
3565886,1,NaN,IBD should sell single issues for the iPad to ...


## Adat előkészítés

* Nan értéket tartalmazó sorok eltávolítása
* Szöveg kisbetűssé alakítása
* Speciális karakterek (pl. írásjelek !,?) eltávolítása
* Számok eltávolítása
* Felesleges szóközök eltávolítása
* Tokenizálás
* Stop szavak eltávolítása
* Lemmatizálás és szótőkeresés

### Nan értéket tartalmazó sorok eltávolítása 

In [33]:
data = data.dropna(subset=['title'])

In [34]:
# Test adat
data_test = data_test.dropna(subset=['title'])

In [35]:
data.shape

(3599793, 3)

### Szöbeg tisztítása regex-el

In [36]:
data['title'] = data['title'].apply(prepare_text)
data['text'] = data['text'].apply(prepare_text)

In [37]:
data.head(10)

,sentiment,title,text
0,2,stuning even for the nongamer,this sound track was beautiful it paints the s...
1,2,the best soundtrack ever to anything,im reading a lot of reviews saying that this i...
2,2,amazing,this soundtrack is my favorite music of all ti...
3,2,excellent soundtrack,i truly like this soundtrack and i enjoy video...
4,2,remember pull your jaw off the floor after hea...,if youve played the game you know how divine t...
5,2,an absolute masterpiece,i am quite sure any of you actually taking the...
6,1,buyer beware,this is a selfpublished book and if you want t...
7,2,glorious story,i loved whisper of the wicked saints the story...
8,2,a five star book,i just finished reading whisper of the wicked ...
9,2,whispers of the wicked saints,this was a easy to read book that made me want...


In [38]:
# Test adat
data_test['title'] = data_test['title'].apply(prepare_text)
data_test['text'] = data_test['text'].apply(prepare_text)

In [39]:
data_test.head(10)

,sentiment,title,text
0,2,great cd,my lovely pat has one of the great voices of h...
1,2,one of the best game music soundtracks for a g...,despite the fact that i have only played a sma...
2,1,batteries died within a year,i bought this charger in jul and it worked ok ...
3,2,works fine but maha energy is better,check out maha energys website their powerex m...
4,2,great for the nonaudiophile,reviewed quite a bit of the combo players and ...
5,1,dvd player crapped out after one year,i also began having the incorrect disc problem...
6,1,incorrect disc,i love the style of this but after a couple ye...
7,1,dvd menu select problems,i cannot scroll through a dvd menu that is set...
8,2,unique weird orientalia from the s,exotic tales of the orient from the s dr shen ...
9,1,not an ultimate guide,firstlyi enjoyed the format and tone of the bo...


### Tokenizálás Stop szavak eltávolítása és Lemmatizálás

In [40]:
data['title_tokens'] = data['title'].apply(tokenizer_stop_words_lemmatize)
data['text_tokens'] = data['text'].apply(tokenizer_stop_words_lemmatize)

In [41]:
# Test adat
data_test['title_tokens'] = data_test['title'].apply(tokenizer_stop_words_lemmatize)
data_test['text_tokens'] = data_test['text'].apply(tokenizer_stop_words_lemmatize)

In [42]:
data.head(10)

,sentiment,title,text,title_tokens,text_tokens
0,2,stuning even for the nongamer,this sound track was beautiful it paints the s...,"[stun, even, nongamer]","[sound, track, beautiful, paint, senery, mind,..."
1,2,the best soundtrack ever to anything,im reading a lot of reviews saying that this i...,"[best, soundtrack, ever, anything]","[im, read, lot, review, say, best, game, sound..."
2,2,amazing,this soundtrack is my favorite music of all ti...,[amaze],"[soundtrack, favorite, music, time, hand, inte..."
3,2,excellent soundtrack,i truly like this soundtrack and i enjoy video...,"[excellent, soundtrack]","[truly, like, soundtrack, enjoy, video, game, ..."
4,2,remember pull your jaw off the floor after hea...,if youve played the game you know how divine t...,"[remember, pull, jaw, floor, hear]","[youve, play, game, know, divine, music, every..."
5,2,an absolute masterpiece,i am quite sure any of you actually taking the...,"[absolute, masterpiece]","[quite, sure, actually, take, time, read, play..."
6,1,buyer beware,this is a selfpublished book and if you want t...,"[buyer, beware]","[selfpublished, book, want, know, whyread, par..."
7,2,glorious story,i loved whisper of the wicked saints the story...,"[glorious, story]","[love, whisper, wicked, saint, story, amaze, p..."
8,2,a five star book,i just finished reading whisper of the wicked ...,"[five, star, book]","[finish, read, whisper, wicked, saint, fell, l..."
9,2,whispers of the wicked saints,this was a easy to read book that made me want...,"[whisper, wicked, saint]","[easy, read, book, make, want, keep, read, eas..."


In [43]:
data.tail(10)

,sentiment,title,text,title_tokens,text_tokens
3599990,2,buy this cd and youll thank yourself,tyler hiltona name you might not know now but ...,"[buy, cd, youll, thank]","[tyler, hiltona, name, might, know, sure, with..."
3599991,2,tyler rocks,there is only one word to describe tyler hilto...,"[tyler, rock]","[one, word, describe, tyler, hiltontalent, lov..."
3599992,2,awesome,absolutely amazing so relieving of my neck pai...,[awesome],"[absolutely, amaze, relieve, neck, pain, inver..."
3599993,1,what a slap in the face to masami ueda,do not buy this cd ever this was probably just...,"[slap, face, masami, ueda]","[buy, cd, ever, probably, release, test, see, ..."
3599994,1,too simplistic,while mr harrison makes some extremely valid a...,[simplistic],"[mr, harrison, make, extremely, valid, argumen..."
3599995,1,dont do it,the high chair looks great when it first comes...,[dont],"[high, chair, look, great, first, come, box, h..."
3599996,1,looks nice low functionality,i have used this highchair for kids now and fi...,"[look, nice, low, functionality]","[use, highchair, kid, finally, decide, sell, l..."
3599997,1,compact but hard to clean,we have a small house and really wanted two of...,"[compact, hard, clean]","[small, house, really, want, two, high, chair,..."
3599998,1,what is it saying,not sure what this book is supposed to be it i...,[say],"[sure, book, suppose, really, rehash, old, ide..."
3599999,2,makes my blood run redwhiteandblue,i agree that every american should read this b...,"[make, blood, run, redwhiteandblue]","[agree, every, american, read, book, everybody..."


### Adatok elmentése csv fájlba

In [ ]:
dest_file = 'cleaned_train.csv'
dest_test_file = 'cleaned_test.csv'

dest_file_path = os.path.join(path, dest_file)
dest_test_file_path = os.path.join(path, dest_test_file)
data.to_csv(dest_file_path)
data_test.to_csv(dest_test_file_path)

: 